In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import edward as ed
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
print(tf.__version__)
from edward.models import Multinomial, Normal, Categorical

1.1.0


In [2]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('../VAE/Datasets/MNIST_data', one_hot=True)
n_train_samples = mnist.train.num_examples
n_test_samples = mnist.test.num_examples
print(n_train_samples)
print(n_test_samples)

Extracting ../VAE/Datasets/MNIST_data/train-images-idx3-ubyte.gz
Extracting ../VAE/Datasets/MNIST_data/train-labels-idx1-ubyte.gz
Extracting ../VAE/Datasets/MNIST_data/t10k-images-idx3-ubyte.gz
Extracting ../VAE/Datasets/MNIST_data/t10k-labels-idx1-ubyte.gz
55000
10000


### Define the model

In [3]:
def forward(X, W_0, b_0, W_1, b_1):
  hidden_layer = tf.nn.relu(tf.matmul(X, W_0) + b_0)
  output = tf.matmul(hidden_layer, W_1) + b_1
  return output

### Priors

In [4]:
num_features = 784
num_classes = 10
hidden_units = 500

W_0 = Normal(loc=tf.zeros([num_features, hidden_units]), scale=tf.ones([num_features, hidden_units]))
b_0 = Normal(loc=tf.zeros(hidden_units), scale=tf.ones(hidden_units))
W_1 = Normal(loc=tf.zeros([hidden_units, num_classes]), scale=tf.ones([hidden_units, num_classes]))
b_1 = Normal(loc=tf.zeros(num_classes), scale=tf.ones(num_classes))

In [5]:
X = tf.placeholder(tf.float32, [None, num_features])
y = Multinomial(logits=forward(X, W_0, b_0, W_1, b_1), total_count=1.)

#y = Categorical(logits=forward(X, W_0, b_0, W_1, b_1))

In [6]:
# Posteriors
qw_0_loc = tf.Variable(tf.random_normal([num_features, hidden_units]))
qw_0_scale = tf.nn.softplus(tf.Variable(tf.random_normal([num_features, hidden_units])))
qb_0_loc = tf.Variable(tf.random_normal([hidden_units]) + 5)
qb_0_scale = tf.nn.softplus(tf.Variable(tf.random_normal([hidden_units])))

qw_0 = Normal(loc=qw_0_loc, scale=qw_0_scale)
qb_0 = Normal(loc=qb_0_loc, scale=qb_0_scale)

qw_1_loc = tf.Variable(tf.random_normal([hidden_units, num_classes]))
qw_1_scale = tf.nn.softplus(tf.Variable(tf.random_normal([hidden_units, num_classes])))
qb_1_loc = tf.Variable(tf.random_normal([num_classes]) + 5)
qb_1_scale = tf.nn.softplus(tf.Variable(tf.random_normal([num_classes])))

qw_1 = Normal(loc=qw_1_loc, scale=qw_1_scale)
qb_1 = Normal(loc=qb_1_loc, scale=qb_1_scale)

In [7]:
y_ph_categorical = tf.placeholder(tf.int32, [None])
y_ph_multinomial = tf.placeholder(tf.float32, [None, num_classes])


inference = ed.KLqp({W_0: qw_0, b_0: qb_0, W_1: qw_1, b_1: qb_1},  data={y: y_ph_multinomial})

In [8]:
inference.initialize(n_iter=1000, n_print=100, scale={y: n_train_samples / 64})

In [9]:
sess = ed.get_session()
tf.global_variables_initializer().run()

In [11]:
for t in range(inference.n_iter):
  xs, ys = mnist.train.next_batch(64)
  
  #ys = np.argmax(ys, axis=1)  #for Categorical
  info_dict = inference.update({X: xs, y_ph_multinomial: ys})
  inference.print_progress(info_dict)

1000/1000 [100%] ██████████████████████████████ Elapsed: 38s | Loss: nan


In [15]:
x_test = mnist.test.images
y_test = mnist.test.labels
logits = forward(x_test, qw_0, qb_0, qw_1, qb_1)
correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(y_test, 1), name="correct_prediction")
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32), name="accuracy")
print("Accuracy:",accuracy.eval())

Accuracy: 0.8553
